# GNN Training - All Datasets with Normalization

**Configuration:**
- Datasets: Cora, PubMed, Roman-empire, Minesweeper
- Model: GCN with `normalize=True` (self-loops enabled)
- K values: 0-8
- Seeds: 0-3
- **Total: 144 models**

**Estimated runtime on A100:** 30-60 minutes

In [ ]:
# Install dependencies (optimized for Colab)
import torch
print(f'Pre-installed PyTorch: {torch.__version__}')
print(f'CUDA: {torch.version.cuda}')

# Install PyTorch Geometric (auto-detects PyTorch version)
import sys
!{sys.executable} -m pip install -q torch-geometric

# Install PyG extensions without pyg-lib (not needed for GCN)
!{sys.executable} -m pip install -q torch-scatter torch-sparse --no-index --find-links https://data.pyg.org/whl/torch-2.5.0+cu128.html

# Install other dependencies
!{sys.executable} -m pip install -q scikit-learn pandas matplotlib seaborn

print('\n✓ All dependencies installed')

# Verify PyG installation
import torch_geometric
print(f'PyTorch Geometric: {torch_geometric.__version__}')

In [ ]:
# Verify GPU
import torch
print(f'GPU available: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'GPU name: {torch.cuda.get_device_name(0)}')
    print(f'GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB')

In [ ]:
# Navigate to project directory
import os
import sys

# Check current directory and find project root
print(f'Starting directory: {os.getcwd()}')
!ls -la

# Try common Colab sync locations
possible_paths = [
    '/content/entropy-selection',
    '/content',
    os.getcwd(),  # Current directory
]

project_dir = None
for path in possible_paths:
    src_path = os.path.join(path, 'src')
    if os.path.exists(src_path) and os.path.isdir(src_path):
        project_dir = path
        break

if project_dir:
    os.chdir(project_dir)
    print(f'\n✓ Found project at: {project_dir}')
    print(f'Current directory: {os.getcwd()}')
    print(f'\nSource modules:')
    !ls src/
else:
    print('\n❌ ERROR: Could not find src/ directory')
    print('Available files in current directory:')
    !ls -la
    raise FileNotFoundError('Project src/ directory not found. Files may not have synced from local.')

In [ ]:
# Triple Training Pipeline
import subprocess
from datetime import datetime

datasets = ['Cora', 'PubMed', 'Roman-empire', 'Minesweeper']
K_values = list(range(9))  # 0-8
seeds = [0, 1, 2, 3]

total_runs = len(datasets) * len(K_values) * len(seeds) * 3  # x3 for 3 pipelines
print(f'Total training runs: {total_runs} (3 pipelines x {len(datasets)} datasets x {len(K_values)} K values x {len(seeds)} seeds)')
print(f'Pipelines: 1) Standard GNN, 2) Exponential classifier heads, 3) Class-weighted classifier heads\n')

current = 0
start_time = datetime.now()

for dataset in datasets:
    for K in K_values:
        for seed in seeds:
            current += 1
            print(f'\n{"="*60}')
            print(f'[{current}/{total_runs}] {dataset} | K={K} | seed={seed}')
            print(f'{"="*60}')
            
            # Pipeline 1: Standard GNN training
            print(f'\n[1/3] Training standard GNN...')
            cmd = f'python -m src.train_gnn --dataset {dataset} --model GCN --K {K} --seed {seed}'
            result = subprocess.run(cmd, shell=True, capture_output=False)
            if result.returncode != 0:
                print(f'❌ Failed: {cmd}')
                continue
            
            # Pipeline 2: Exponential classifier heads
            current += 1
            print(f'\n[{current}/{total_runs}] [2/3] Training exponential classifier heads...')
            cmd = f'python -m src.train_gnn_classifier_heads --dataset {dataset} --model GCN --K {K} --seed {seed} --loss-type exponential --beta 0.5'
            result = subprocess.run(cmd, shell=True, capture_output=False)
            if result.returncode != 0:
                print(f'❌ Failed: {cmd}')
            
            # Pipeline 3: Class-weighted classifier heads
            current += 1
            print(f'\n[{current}/{total_runs}] [3/3] Training class-weighted classifier heads...')
            cmd = f'python -m src.train_gnn_classifier_heads --dataset {dataset} --model GCN --K {K} --seed {seed} --loss-type class-weighted --beta 0.5'
            result = subprocess.run(cmd, shell=True, capture_output=False)
            if result.returncode != 0:
                print(f'❌ Failed: {cmd}')
            
            # Progress update
            elapsed = (datetime.now() - start_time).total_seconds() / 60
            avg_time_per_triple = elapsed / (current / 3)
            remaining = (total_runs / 3 - current / 3) * avg_time_per_triple
            print(f'\n⏱  Elapsed: {elapsed:.1f}min | Est. remaining: {remaining:.1f}min')

print(f'\n{"="*60}')
print(f'✓ ALL TRAINING COMPLETE!')
total_time = (datetime.now() - start_time).total_seconds() / 60
print(f'Total time: {total_time:.1f} minutes')
print(f'{"="*60}')


In [ ]:
# Training complete!
print('✓ Training complete!')
print('Results saved in results/runs/')
print('\nNext steps (run locally):')
print('1. Extract embeddings')
print('2. Run probing')
print('3. Generate plots')